In [1]:
import tokenizers
from tokenizers import Tokenizer
import os, time, re,json, copy
from collections import OrderedDict
from statistics import fmean

In [4]:
base_tokenizer_fn = "/home/karyo/corpus/models/04_27_camel/unigram/kr_64k/tokenizer.json"
plus_tokenizer_fn = "/home/karyo/corpus/models/04_27_camel/unigram/ec_64k/tokenizer.json"
final_json_fn = "/home/karyo/corpus/models/union/tokenizer.json"

In [5]:
base_json = json.load(open(base_tokenizer_fn))
plus_json = json.load(open(plus_tokenizer_fn))

In [53]:
target = 102400
whitespace_reservation = 24 - 1 # skip single space
def get_counts(tokenizer_json : dict) -> tuple:
    added_token_count = len(tokenizer_json['added_tokens'])
    vocab_size = len(tokenizer_json['model']['vocab'])
    return (added_token_count, vocab_size)

In [54]:
# acquire token counts from tokenizers
base_added_token_count, base_vocab_size = get_counts(base_json)
plus_added_token_count, plus_vocab_size = get_counts(base_json)

# how much merges to get for final target
target_tokens = target - whitespace_reservation

In [55]:
base_added_token_count

131

In [56]:
# we will operate on the values so deep copy
base_vocab = copy.deepcopy(base_json['model']['vocab'])
plus_vocab = copy.deepcopy(plus_json['model']['vocab'])

In [57]:
base_vocab_dict = {vocab_tuple[0] : vocab_tuple[1] for vocab_tuple in base_vocab}
plus_vocab_dict = {vocab_tuple[0] : vocab_tuple[1] for vocab_tuple in plus_vocab}

In [58]:
print(len(base_vocab_dict), len(plus_vocab_dict))

65513 65536


In [59]:
token_count = 0
union_vocab_dict = base_vocab_dict.copy()
union_vocab_keys = union_vocab_dict.keys()
for token in plus_vocab_dict.keys():
    if token not in union_vocab_keys:
        union_vocab_dict[token] = plus_vocab_dict[token]
    if token in union_vocab_keys:
        log_probs = [union_vocab_dict[token],plus_vocab_dict[token]]
        union_vocab_dict[token] = fmean(log_probs)

In [60]:
sorted_union_list = sorted(union_vocab_dict.items(),key=lambda x:x[1],reverse=True)[:target_tokens]
sorted_union_dict = dict(sorted_union_list)

In [66]:
# copy original base json to add to
final_json = copy.deepcopy(base_json)
#inverse_final_json = {index : vocab for vocab,index in final_json['model']['vocab'].items()}
final_json['model']['vocab'] = sorted_union_list
final_json['model']['unk_id'] = 5

In [67]:
len(final_json['model']['vocab'])

102377

In [68]:
with open(final_json_fn, "w") as output_f:
    json.dump(final_json, output_f, indent=2, sort_keys=False, ensure_ascii=False)

In [69]:
final_tokenizer = Tokenizer.from_file(final_json_fn)

In [70]:
print(final_tokens := final_tokenizer.get_vocab_size(with_added_tokens=True), target, final_tokens == target)

102400 102400 True


In [71]:
final_tokenizer.save(final_json_fn)